# COGS 108 - Final Project 

## Permissions

Place an `X` in the appropriate bracket below to specify if you would like your group's project to be made available to the public. (Note that PIDs will be scraped from the public submission, but student names will be included.)

* [  ] YES - make available
* [ x ] NO - keep private

# Overview

*Fill in your overview here*

# Names

- Yang Li
- Yiou Lyu
- Linfeng Hu
- Ruby Celeste Marroquin 

# Group Members IDs

- A15560579
- A15930345
- A15473121
- A16094382

# Research Question

How does the regional economic status of each province in mainland China correlate to its breakout and recovery of COVID-19?

## Background and Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


*Fill in your hypotheses here*

# Dataset(s)

(Copy this information for each dataset)
- Dataset Name: 
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [1]:
import pandas as pd
import json
import codecs
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import os
import patsy
import scipy.stats as stats

import bs4
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

# Data Cleaning

In [2]:
#with codecs.open('Data/virus.json', 'r', 'utf-8') as data_file:
    #data_teacher = json.load(data_file, 'utf-8')

#topic[worksheet] = data_teacher[worksheetID]['Topic']
#out = codecs.open('Worksheet.csv', 'w', 'utf-8')
#out.write(topic[worksheet])

To clean our data, our general approach is to represent datasets in pandas dataframe. Then we drop irrelevant information or outliers in data. We also rename the columns to make it easier for later analyses.

Firstly, we deal with the datasets that consist of economic status data. 

This is the income per capita value for each province in mainland China. Income is measured in yuan. 

In [3]:
Income = pd.read_csv('Data/Income.csv')
Income = Income.dropna(axis=1, how='all')
Income.head()

FileNotFoundError: [Errno 2] File b'Data/Income.csv' does not exist: b'Data/Income.csv'

This is the per capita Gross Regional Product value for each province. GRP per capita is measured in yuan.

In [ ]:
GRP = pd.read_csv('Data/GRP.csv')
GRP = GRP.dropna(axis=1, how='all')
GRP.head()

Next, we move on to clean the population density related datasets.

Population per province here is calculated in the unit of 10000 persons). It includes all residents (permanent and temporary, rural and urban)at the end of that year.

In [ ]:
population = pd.read_csv('Data/Population.csv')
population = population.dropna(axis = 1, how = 'all')
population.head()

To calculate population density of a region, we also need to areas of each province. Here, area of each province is measured in unit of square kilometers.

Since we only need the area information of each separate region, we will drop the "Toal" row at the end which contains information about the total area of China(judging by the data contained, the row name should be a typo).

In [ ]:
area = pd.read_csv('Data/Area.csv')
area = area.dropna(axis = 1, how = 'all')
#shorten column names to make following analysis simpler
area = area.rename(columns={"Area (sq.km)": "Area"})
area = area[area.District != 'Toal']
area.head()

In [ ]:
# read virus data into dataframes 

list_of_virus_data = list()

# append data between Feb 1 and Feb 25 to list
for i in range(20200201,20200226): 
    path = './Data/virus/' + str(i) + '.csv'
    list_of_virus_data.append(pd.read_csv(path))
    
# File 20200226.csv is missing, reason unknow. 

    
# append data between Feb 27 and Feb 29 to list
for i in range(20200227,20200230): 
    path = './Data/virus/' + str(i) + '.csv'
    list_of_virus_data.append(pd.read_csv(path))

# append data between Mar 1 and  Mar 1 to list
for i in range(20200301,20200302): 
    path = './Data/virus/' + str(i) + '.csv'
    list_of_virus_data.append(pd.read_csv(path))
    
print('number of dataframes for virus: ',len(list_of_virus_data))

# access ith elment in the list using list_of_virus_data[i]
# for example list_of_virus_data[0] gives the first dataframe


## Start cleaning virus data

### Clean 0th to 1th df in the list 

In [ ]:
# Clean 0th to 1th df in the list 
for i in range(0,2):
    # get the df of the ith day
    df = list_of_virus_data[i]
    # use the first data row as column names
    df.columns = df.iloc[0]
    # drop first row, because is was used as header
    df = df.drop(0)
    # drop the column '1', because it is irrelevant
    df = df.drop(1, axis=1)
    # save cleaned data to list_of_virus_data 
    list_of_virus_data[i] = df

### Clean 2th df in the list

In [ ]:
# Clean 2th df in the list
# get the df of the ith day
df = list_of_virus_data[2]
# reset column names
df.columns = ["Province/Region/City", "Confirmed Cases", 1]
# drop meaningless 1" column,  keep "Confirmed Cases" and "Province/Region/City"
df = df.drop(1, axis=1)
# Drop the last row, because it is comment instaed of data
df = df.drop(df.shape[0] - 1)
# save cleaned data to list_of_virus_data 
list_of_virus_data[2] = df

### Clean 3th to 10th df in the list 

In [ ]:
# Clean 3th to 10th df in the list 
for i in range(3,11):
    # get the df of the ith day
    df = list_of_virus_data[i]
    # use the first data row as column names
    df.columns = df.iloc[0]
    # drop first row, because is was used as header
    df = df.drop(0)
    # drop the column '1', because it is irrlavent
    df = df.drop(1, axis=1)
    # save cleaned data to list_of_virus_data
    list_of_virus_data[i] = df

In [ ]:
# View virus data of the first 11 days
# Delete this cell
for i in range (0,11):
    print(list_of_virus_data[i])
    print("\n")
    print("\n")
    print("\n")

In [ ]:
# Clean 11th to the 28th df in the list 
# TODO

In [4]:
def fix_columns(df):
    
    #Get a list of the columns 
    data = list(df.columns)
    
    #Drops the number of records 
    if(len(data) <= 4):
        df['Suspected Cases'] = 0
        df['Deaths'] = 0
        df.drop(df.columns[3], axis=1, inplace=True)
    if(len(data) > 4):
        df.drop(df.columns[2], axis=1, inplace=True)
        df.drop(df.columns[len(data)-1], axis=1, inplace=True)
        df['Suspected Cases'] = 0
        df['Deaths'] = 0
    
    df = df[['Date', 'Province/Region/City', 'Confirmed Cases',
                 'Suspected Cases', 'Deaths']]
    
    df.drop(df.columns[0], axis=1, inplace=True)
    df.drop(df.tail(1).index,inplace=True)

    
    return df

In [5]:
def fix_columns_2(df):
    
    #Get a list of the columns 
    data = list(df.columns)
    
    #drops the number of records column 
    if "Number of Records" in data:
        df.drop(df.columns[len(data)-1], axis=1, inplace=True)
    if(len(data) >= 12):
        df.drop(df.columns[[2,3, 4, 5, 7, 9,10]], axis=1, inplace=True)
        if "Lab-confirmed 1" in data:
            df.rename(columns={"Lab-confirmed 1":'Confirmed Cases'}, inplace=True)
        if "Deaths 1" in data: 
            df.rename(columns={"Deaths 1":'Deaths'}, inplace=True)
    if(len(data) < 12):
        #used to remove population 
        if "Population (in 10,000s)" in data:
            df.drop(df.columns[2], axis=1, inplace=True)
        
        #Renames total deaths to deaths 
        if "Total Deaths" in data:
            df.rename(columns={'Total Deaths':'Deaths'}, inplace=True)
    
        #Checks to see if suspected cases and/or death exists or else creates
        #and fills with 0 since no data was available at the time 
        if not "Suspected Cases" in data:
            df['Suspected Cases'] = 0
        if not "Deaths" in data:
            df['Deaths'] = 0
    
    #Drops any remaining columns that aren't any of the following
    df = df[['Date', 'Province/Region/City', 'Confirmed Cases',
             'Suspected Cases', 'Deaths']]
    df.drop(df.columns[0], axis=1, inplace=True)
    df.drop(df.tail(1).index,inplace=True)
    
    return df

In [6]:
def fix_columns_3(df):
    #Get a list of the columns 
    data = list(df.columns)
    
    #deletes the number of records column 
    del df['Number of Records']
    
    if "F5" in data:
        del df['F5']
    if "F3" in data:
        del df['F3']
    if "F2" in data:
        del df['F2']
    if 'Population' in data:
        del df['Population']
    if 'Population (in 10,000s)' in data:
        del df['Population (in 10,000s)']
    if 'F1' in data:
        df.rename(columns={'F1':'Province/Region/City'}, inplace=True)
    if 'F4' in data:
        df.rename(columns={'F4':'Suspected Cases'}, inplace=True)
    if 'F6' in data:
        df.rename(columns={'F6':'Confirmed Cases'}, inplace=True)
    if 'F7' in data:
        df.rename(columns={'F7':'Deaths'}, inplace=True)
    
    df = df[['Province/Region/City', 'Confirmed Cases',
        'Suspected Cases', 'Deaths']]
    
    df.drop(df.index[[0, 1, 2, 3]], inplace=True)
    
    df.drop(df.tail(1).index,inplace=True)
    df.dropna(how='any')
    
    return df

In [7]:
files_to_read = ["Feb1_Data.csv", "Feb2_Data.csv", "Feb3_Data.csv", "Feb4_Data.csv", "Feb5_Data.csv",
                 "Feb6_Data.csv", "Feb7_Data.csv", "Feb8_Data.csv", "Feb9_Data.csv", "Feb10_Data.csv", 
                 "Feb11_Data.csv", "Feb12_Data.csv", "Feb13_Data.csv", "Feb14_Data.csv", "Feb15_Data.csv",
                 "20200216.csv", "20200217.csv", "20200218.csv", "20200219.csv", "20200220.csv", 
                 "20200221.csv", "20200222.csv", "20200223.csv", "20200224.csv", "20200225.csv", "20200227.csv",
                 "20200228.csv", "20200229.csv", "20200301.csv"]

In [8]:
#Creates a new DataFrame with the first file and used to concat the rest 
df = pd.read_csv("Feb1_Data.csv")
df = fix_columns(df)

for i in range(0, 10):
    next_file = pd.read_csv(files_to_read[i])
    next_file = fix_columns(next_file)
    df = pd.concat([df, next_file], axis = 0, join = 'inner')

In [9]:
for i in range(11, 15):
    next_file = pd.read_csv(files_to_read[i])
    next_file = fix_columns_2(next_file)
    df = pd.concat([df, next_file], axis = 0, join = 'inner')

In [10]:
for i in range(16, 29):
    next_file = pd.read_csv(files_to_read[i])
    if(i == 21):
        next_file.columns = ['Population', 'F3', 'Suspected Cases', 'F5', 'Confirmed Cases', 'Deaths', 
                     'Number of Records', 'Province/Region/City']
        next_file = fix_columns_3(next_file)
    elif (i == 25):
        next_file.columns = ['Population', 'F3', 'Suspected Cases', 'F5','Confirmed Cases', 'Deaths', 
                     'Number of Records', 'Province/Region/City']
        next_file = fix_columns_3(next_file)
    else:    
        next_file = fix_columns_3(next_file)
    
    df = pd.concat([df, next_file], axis = 0, join = 'inner')
        

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [ ]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*

# Team Contributions

*Specify who in your group worked on which parts of the project.*